In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
sns.set()

os.chdir('../..')

DATA = Path('data')
RAW  = DATA/'raw'
PROCESSED = DATA/'processed'
SUBMISSIONS = DATA/'submissions'

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)

In [3]:
%%time
categories = pd.read_csv(RAW/'productid_category.csv', low_memory=False)
train_tracking = pd.read_csv(RAW/'train_tracking.csv', low_memory=False)
test_tracking  = pd.read_csv(RAW/'test_tracking.csv', low_memory=False)
train_session = pd.read_csv(RAW/'train_session.csv', low_memory=False)
test_session  = pd.read_csv(RAW/'random_submission.csv', low_memory=False)

CPU times: user 1min 6s, sys: 6.63 s, total: 1min 13s
Wall time: 1min 13s


In [2]:
nrows = 30
ncatcols = 4
ncontcols = 5
seqlen = 10
ncatseqcols = 2
ncontseqcols = 3
batch_size = 5

def get_set(nrows):
    cats = np.random.randint(0, 40, (nrows, ncatcols))
    conts = np.random.rand(nrows, ncontcols)
    catseqs = np.random.randint(0, 40, (nrows, seqlen, ncatseqcols))
    contseqs = np.random.rand(nrows, seqlen, ncontseqcols)
    targets = np.random.randint(0, 2, (nrows))
    return MultimodalDataset(cats, conts, catseqs, contseqs, targets)

In [4]:
from src.utils_r import MultimodalDataset
from torch.utils.data import DataLoader
ds_trn = get_set(200)
ds_val = get_set(40)

In [5]:
dl_trn = DataLoader(ds_trn, batch_size=5)
dl_val = DataLoader(ds_val, batch_size=5)

In [6]:
from src.utils_r import MultimodalNet

In [7]:
emb_szs = [(45, 2), (50, 3), (43, 2), (42, 5)]
n_cont = ncontcols
emb_drop = 0.3
szs = [200, 100]
drops = [0.1, 0.1]

rnn_hidden_sz = 15
rnn_cont_sz = ncontseqcols
rnn_emb_sz = [(41, 10), (50, 4)]
rnn_n_layers = 2
rnn_drop = 0.2
rnn_out_sz = 1

In [8]:
model = MultimodalNet(emb_szs, n_cont, emb_drop, szs, drops, 
                      rnn_hidden_sz, rnn_cont_sz, rnn_emb_sz, 
                      rnn_n_layers, rnn_drop, rnn_out_sz)

In [13]:
from src.utils_r import train_model
from torch import optim
import torch.nn as nn


In [14]:
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.BCEWithLogitsLoss()

In [21]:
USE_CUDA = True

In [22]:
model = MultimodalNet(emb_szs, n_cont, emb_drop, szs, drops, 
                      rnn_hidden_sz, rnn_cont_sz, rnn_emb_sz, 
                      rnn_n_layers, rnn_drop, rnn_out_sz)
if USE_CUDA: model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.BCEWithLogitsLoss()

In [23]:
best_epoch = train_model(model, dl_trn, dl_val, optimizer, criterion, 10, USE_CUDA=USE_CUDA)

Epoch: 1 | Train Logloss: 0.69 | Val Logloss: 0.72
Epoch: 2 | Train Logloss: 0.69 | Val Logloss: 0.72
Epoch: 3 | Train Logloss: 0.69 | Val Logloss: 0.72
Epoch: 4 | Train Logloss: 0.69 | Val Logloss: 0.72
Epoch: 5 | Train Logloss: 0.68 | Val Logloss: 0.72
Epoch: 6 | Train Logloss: 0.68 | Val Logloss: 0.72
Epoch: 7 | Train Logloss: 0.68 | Val Logloss: 0.73
Epoch: 8 | Train Logloss: 0.67 | Val Logloss: 0.73
Epoch: 9 | Train Logloss: 0.67 | Val Logloss: 0.72
Epoch: 10 | Train Logloss: 0.67 | Val Logloss: 0.73


In [24]:
from src.utils_r import eval_model
y_val, y_val_pred = eval_model(model, dl_val, USE_CUDA=USE_CUDA)

In [25]:
from sklearn.metrics import log_loss
log_loss(y_val, y_val_pred)

0.7321644017554069